# Compare Brazilian States Trajectories
> Cases and Deaths - Accumulated (linear scale)

- toc: true
- badges: true
- comments: true
- author: Cleber Jorge Amaral, based on [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html) work published at [covid19-dashboard](https://github.com/github/covid19-dashboard) which uses [fastpages](https://github.com/fastai/fastpages) by [Hamel Husain](https://github.com/hamelsmu).
- categories: [jupyter]
- image: images/brazil-states-linear-case-and-deaths-accumulated.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
# Get source original file from brasil.io
# Expected to have the columns: ['state', 'date', 'confirmed']
url = ('https://data.brasil.io/dataset/covid19/caso.csv.gz')
src = pd.read_csv(url)

In [3]:
#hide
STATE_COLUMN = "Province/State"
DATE_COLUMN = "Date"
CONFIRMED_COLUMN = "Confirmed"
DEATH_COLUMN = "Death"
NEWCONFIRMED_COLUMN = "New_Confirmed"
NEWDEATH_COLUMN = "New_Death"
NEWRECOVERY_COLUMN = "New_Recovery"
ACTIVE_COLUMN = "Active"


In [4]:
#hide
df = src[(src['place_type'] == 'state')]
df = df.rename(columns={"state": STATE_COLUMN})
df = df.rename(columns={"date": DATE_COLUMN})
df = df.rename(columns={"confirmed": CONFIRMED_COLUMN})
df = df.rename(columns={"deaths": DEATH_COLUMN})
#df.tail()

In [5]:
#hide_input
input_dropdown = alt.binding_select(options=df[STATE_COLUMN].unique())
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(CONFIRMED_COLUMN, axis=alt.Axis(title=CONFIRMED_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, CONFIRMED_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title='Total confirmed cases by state in Brazil'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend

alt.HConcatChart(...)

In [6]:
#hide_input
input_dropdown = alt.binding_select(options=df[STATE_COLUMN].unique())
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(DEATH_COLUMN, axis=alt.Axis(title=DEATH_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, STATE_COLUMN, DEATH_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title='Total deaths by state in Brazil'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend

alt.HConcatChart(...)

Based on works developed by [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html), adapted by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). Data source: [brasil.io](https://brasil.io/home/)